<a href="https://colab.research.google.com/github/HodaMemar/A3/blob/main/Pivot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import exists


In [ ]:
filename="/content/drive/MyDrive/PaperA3/CUI__0.csv"
df=pd.read_csv(filename)
for i in range(1,76):
  filename="/content/drive/MyDrive/PaperA3/CUI__"+str(i)+".csv"
  if (exists(filename)==True):
          print(i)
          df1=pd.read_csv(filename)
          df=pd.concat([df, df1])

In [ ]:
df.to_csv('/content/drive/MyDrive/PaperA3/CUI_Total.csv')

In [ ]:
def fxy(x, y):
        if(x =='True'):
            r='not ' + y 
        else:
            r=y
        return r             

#Pivot and integrating primary dataset

In [ ]:
%%time
for k in range(0,76):
  filename="/content/drive/MyDrive/PaperA3/CUI__"+str(k)+".csv"
  if (exists(filename)==True):
          
          df=pd.read_csv(filename)
          #df=pd.concat([df, df1])
          df['Neg_CUI'] = df.apply(lambda x: fxy(x['negex'], x['canonical_name']), axis=1)
          ls_patient=df.HADM_ID.unique()
          print("CUI__"+str(k) + "_Number of Patients:  " + str(len(ls_patient)) + ",   Number of records:  " + str(len(df)))
          FormName=''

          ls_Category_Section=[]

          for i in range(len(ls_patient)):
            ls_category=df[(df['HADM_ID']==ls_patient[i] )].category_Inner.dropna().unique().tolist()
            print(i,ls_patient[i])
            for j in  range(len(ls_category)):
                FormName=df[(df['HADM_ID']==ls_patient[i] ) & (df['category_Inner']==ls_category[j] )].CATEGORY.unique()
                #print(FormName)
                ls_Category_Section.append([k,ls_patient[i],ls_category[j]
                                            ,FormName
                                            ,df[(df['HADM_ID']==ls_patient[i] ) & (df['category_Inner']==ls_category[j] )].first_cuid.values.tolist()
                                            ,df[(df['HADM_ID']==ls_patient[i] ) & (df['category_Inner']==ls_category[j] )].canonical_name.values.tolist()
                                            ,df[(df['HADM_ID']==ls_patient[i] ) & (df['category_Inner']==ls_category[j] )].Neg_CUI.values.tolist()])

          dt=pd.DataFrame(ls_Category_Section,columns=['index','HADM_ID','category_Inner','FormName','CUI','canonical_name','Neg_CUI'])
          dt.to_csv('/content/drive/MyDrive/PaperA3/Processed/processed_'+str(k)+'.csv')

In [ ]:
ls_patient=dt.HADM_ID.unique()
print("Number of Patients:  " + str(len(ls_patient)) + ",   Number of records:  " + str(len(dt)))